In [29]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from scipy.stats import f_oneway
import re
sns.set()

# Loading Essay Score Dataset & perfom NLP Data Preprocessing

In [24]:
data_path = "data/train.csv"
df = pd.read_csv(data_path)

data_path = "data/test.csv"
df_test = pd.read_csv(data_path)

In [28]:
X = df.drop(columns=["score"])
y = df["score"]


## NLP Preprocessing

In [30]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)


cList = {#"dont" : "do not", "doesnt" : "does not", "thats" : "that is"
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would",  ## --> he had or he would
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is", 
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",   ## --> I had or I would
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",   ## --> It had or It would
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",   ## --> It had or It would
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
}
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    #x = re.sub("'\d+", '',x)
    #x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Remove \xa0
    x = x.replace(u'\xa0',' ')
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    x = expandContractions(x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
#     x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
  #replace \'s with 's
    #print(re.findall("\\'s", x))
    #x = re.sub(r"\[\]'s", "'s", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

<>:59: SyntaxWarning: invalid escape sequence '\w'
<>:64: SyntaxWarning: invalid escape sequence '\w'
<>:59: SyntaxWarning: invalid escape sequence '\w'
<>:64: SyntaxWarning: invalid escape sequence '\w'
C:\Users\kevin\AppData\Local\Temp\ipykernel_37408\2985774794.py:59: SyntaxWarning: invalid escape sequence '\w'
  x = re.sub("@\w+", '',x)
C:\Users\kevin\AppData\Local\Temp\ipykernel_37408\2985774794.py:64: SyntaxWarning: invalid escape sequence '\w'
  x = re.sub("http\w+", '',x)


In [31]:
X["full_text"] = X.apply(lambda x: dataPreprocessing(x["full_text"]),axis=1)

In [33]:
X.drop(columns=["essay_id"],inplace=True)

# Fine Tuning Llama-3 using PERFT QLORA

In [2]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub


In [6]:
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [7]:


# Download latest version
path = kagglehub.model_download("metaresearch/llama-3/pyTorch/8b")

print("Path to model files:", path)

100%|██████████| 11.9G/11.9G [06:29<00:00, 32.8MB/s]  
Extracting model files...


Path to model files: C:\Users\kevin\.cache\kagglehub\models\metaresearch\llama-3\pyTorch\8b\1


In [21]:
PATH_TO_LLAMA3 = "C:/Users/kevin/.cache/kagglehub/models/metaresearch/llama-3/pyTorch/8b/1"

In [22]:
import config

In [23]:
exp_config = config.Config()

# Adapt Essay Scoring DataFrame to be compatible with Pytorch

In [ ]:
%pip install transformers

In [ ]:
from dataclasses import dataclass
from torch.utils.data import DataLoader, Dataset
from typing import Optional, Union, Any
from transformers import DataCollatorWithPadding


from transformers import AutoTokenizer

def define_tokenizer(cfg):
    """
    Let's use basic AutoTokenizer
    """

    tokenizer = AutoTokenizer.from_pretrained(cfg.architecture["backbone"], trust_remote_code=True)

    # Make sure that we have a pad token and and eos token that will be used for pooling
    if tokenizer.pad_token is None:
        print("Setting new pad token")
        # pad token is missig        
        tokenizer.pad_token="<|reserved_special_token_0|>"
        
    if tokenizer.eos_token is None:
        print("Setting new eos_token token")
        # eos_token token is missig
        tokenizer.eos_token="<|reserved_special_token_1|>"
    
    # Make sure that padding is always "right"
    if tokenizer.padding_side != "right":
        print(f"Changing padding side from {tokenizer.padding_side} to 'right'")
        tokenizer.padding_side = "right"
    return tokenizer